### スクレイピング

In [192]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
import re  # 数字抽出用のライブラリ

# 初期設定
base_url = (
    "https://www.jalan.net/theme/ski/niigata/?roomCount=1&ssc=126195&adultNum=2&distCd=05&"
    "stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&"
    "screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195&"
)
results = []  # 取得結果を保存
page_count = 0  # ページカウント
items_per_page = 30  # 1ページの表示件数
max_pages = 50  # 最大ページ制限
idx = 0  # 最初のページのインデックス

while page_count < max_pages:
    # 動的にURLを構築
    full_url = f"{base_url}idx={idx}#"
    print(f"取得中: {full_url}")

    # ページを取得
    response = requests.get(full_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # 件数の確認（最初のページだけで十分）
        if page_count == 0:
            total_count_tag = soup.select_one('.jlnpc-listInformation--count')
            total_count = int(total_count_tag.text.strip()) if total_count_tag else 0
            print("総件数:", total_count)

        # 宿泊施設データを取得
        items = soup.select('.p-yadoCassette__summary')  # 宿泊施設のリストを取得
        for item in items:
            hotel_name_tag = item.select_one('.p-searchResultItem__facilityName')
            price_tag = item.select_one('.p-searchResultItem__lowestPriceValue')
            rating_tag = item.select_one('.p-searchResultItem__summaryaverage-num')
            highly_rated_tag = item.select_one('.p-searchResultItem__highlyRated strong')

            hotel_name = hotel_name_tag.text.strip() if hotel_name_tag else ""
            price = price_tag.text.strip() if price_tag else ""
            price = re.sub(r'[^\d]', '', price)  # 数字以外を削除
            price = int(price) // 2 if price else 0  # 数値が存在する場合は 1/2 に変換

            rating = rating_tag.text.strip() if rating_tag else ""
            highly_rated = highly_rated_tag.text.strip() if highly_rated_tag else ""

            results.append({
                "hotel_name": hotel_name,
                "price": price,
                "rating": rating,
                "highly_rated": highly_rated,
            })

        # ページカウントと次のインデックスを更新
        page_count += 1
        idx += items_per_page

        # 次のインデックスが総件数を超えたら終了
        if idx >= total_count:
            break

        print(f"ページ {page_count} を取得しました。")
        time.sleep(1)  # サーバー負荷軽減のため1秒待機
    else:
        print(f"ページ {page_count + 1} の取得に失敗しました。")
        break

# 結果の出力
#for idx, result in enumerate(results, 1):
    #print(f"{idx}. hotel_name: {result['hotel_name']}, price: {result['price']}円〜, rating: {result['rating']}, highly_rated: {result['highly_rated']}")



# 件数確認
print(f"\n取得件数: {len(results)} 件")
if total_count and len(results) == total_count:
    print("すべてのデータを取得しました。")
else:
    print("一部データが取得できていません。")

# データベースを作成・更新
create_table_sql = """
CREATE TABLE IF NOT EXISTS hotels (
    hotel_name TEXT,
    price INTEGER,
    rating TEXT,
    highly_rated TEXT
)
"""

# データベースに接続
conn = sqlite3.connect("jalan_hotels.db")
c = conn.cursor()

# テーブルを作成
c.execute(create_table_sql)

# テーブルのデータをクリア
c.execute("DELETE FROM hotels")
print("既存データを削除しました。")

# データを挿入
insert_sql = "INSERT INTO hotels (hotel_name, price, rating, highly_rated) VALUES (?, ?, ?, ?)"
for result in results:
    c.execute(insert_sql, (result['hotel_name'], int(result['price']) if result['price'] else None, result['rating'], result['highly_rated']))

# コミットしてクローズ
conn.commit()
conn.close()
print("新しいデータをデータベースに保存しました。")

# データベースを読み込んで出力
conn = sqlite3.connect("jalan_hotels.db")
c = conn.cursor()
c.execute("SELECT * FROM hotels")
rows = c.fetchall()

for idx, row in enumerate(rows, 1):
    print(f"{idx}. hotel_name: {row[0]}, price: {row[1]}円〜, rating: {row[2]}, highly_rated: {row[3]}")

conn.close()
print("データベースのデータを出力しました。")


取得中: https://www.jalan.net/theme/ski/niigata/?roomCount=1&ssc=126195&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195&idx=0#
総件数: 149
ページ 1 を取得しました。
取得中: https://www.jalan.net/theme/ski/niigata/?roomCount=1&ssc=126195&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195&idx=30#
ページ 2 を取得しました。
取得中: https://www.jalan.net/theme/ski/niigata/?roomCount=1&ssc=126195&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195&idx=60#
ページ 3 を取得しました。
取得中: https://www.jalan.net/theme/ski/niigata/?roomCount=1&ssc=126195&adultNum=2&distCd=05&stayYear=&stayMonth=&dateUndecided=1&roomCrack=200000&vosFlg=6&mvTabFlg=1&stayDay=&screenId=UWW1402&activeSort=0&stayCount=1&rootCd=01126195

### 分析